<a href="https://colab.research.google.com/github/NithickRoshan/githupbemc2/blob/main/NER_Model_for_FRA_records.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import spacy

In [ ]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 4.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
nlp=spacy.load("en_core_web_lg")
nlp

In [ ]:
doc=nlp("Village Thiruvallur Patta ORG holder Ramesh Kumar PERSON Coordinates 12.9345 CARDINAL , 80.2123 Status Disputed")

In [ ]:
doc.ents

(Village Thiruvallur Patta ORG, Ramesh Kumar, 12.9345, CARDINAL)

In [ ]:
doc.ents[0],type(doc.ents[0])

(Village Thiruvallur Patta ORG, spacy.tokens.span.Span)

In [ ]:
from spacy import displacy
displacy.render(doc, style="ent",jupyter=True)

In [ ]:
import json
with open('/content/NER dataset.json', 'r') as f:
    data=json.load(f)

In [ ]:
data[0]

{'id': 1,
 'text': 'Salem village patta holder is Ramesh Kumar at 11.659,78.146. Claim status: Pending.',
 'entities': [{'start': 0, 'end': 5, 'label': 'VILLAGE', 'text': 'Salem'},
  {'start': 30, 'end': 42, 'label': 'PATTA_HOLDER', 'text': 'Ramesh Kumar'},
  {'start': 46, 'end': 59, 'label': 'COORDINATE', 'text': '11.659,78.146'},
  {'start': 75, 'end': 82, 'label': 'CLAIM_STATUS', 'text': 'Pending'}]}

In [ ]:
data[0].keys()

dict_keys(['id', 'text', 'entities'])

In [ ]:
data[0]['entities'][0]

{'start': 0, 'end': 5, 'label': 'VILLAGE', 'text': 'Salem'}

In [ ]:
data[0]['text']

'Salem village patta holder is Ramesh Kumar at 11.659,78.146. Claim status: Pending.'

In [ ]:
training_data = []
for example in data:
    temp_dict = {}
    temp_dict['text'] = example['text']
    temp_dict['entities'] = []
    for annotation in example['entities']:
        start = annotation['start']
        end = annotation['end']
        label = annotation['label'].upper()
        temp_dict['entities'].append((start, end, label))
    training_data.append(temp_dict)

print(training_data[0])

{'text': 'Salem village patta holder is Ramesh Kumar at 11.659,78.146. Claim status: Pending.', 'entities': [(0, 5, 'VILLAGE'), (30, 42, 'PATTA_HOLDER'), (46, 59, 'COORDINATE'), (75, 82, 'CLAIM_STATUS')]}


In [ ]:
training_data[0]['text']


'Salem village patta holder is Ramesh Kumar at 11.659,78.146. Claim status: Pending.'

In [ ]:
training_data[0]['entities']

[(0, 5, 'VILLAGE'),
 (30, 42, 'PATTA_HOLDER'),
 (46, 59, 'COORDINATE'),
 (75, 82, 'CLAIM_STATUS')]

In [ ]:

training_data[0]['text'][360:371]

''

In [ ]:
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en")  # load a new spacy model
doc_bin = DocBin()

In [ ]:
from spacy.util import filter_spans

for training_example in tqdm(training_data):
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents
    doc_bin.add(doc)

doc_bin.to_disk("train.spacy")

100%|██████████| 20/20 [00:00<00:00, 1890.94it/s]

Skipping entity


In [ ]:
!python -m spacy init config base_config.cfg --lang en --pipeline ner

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
base_config.cfg
You can now add your data and train your pipeline:
python -m spacy train base_config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg


⚠ Nothing to auto-fill: base config is already complete
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     55.22    0.00    0.00    0.00    0.00
 66     200         21.49   1422.27  100.00  100.00  100.00    1.00
150     400          0.00      0.00  100.00  100.00  100.00    1.00
250     600          0.00      0.00  100.00  100.00  100.00    1.00
350     800          0.00      0.00  100.00  100.00  100.00    1.00
508    1000          0.65      0.39  100.00  100.00  100.00    1.00
708    1200         13.85      6.80  100.00  100.00  100.00    1.00
908    1400          0.00      0.00  100.00  100.00  100.00    1.00
1108    1600          3.83      0.93

In [ ]:
import spacy

# Load the trained model
nlp_ner = spacy.load('./model-best')

doc = nlp_ner("Village Chengalpattu Coordinates 12.6929, 79.9771 listed under Patta holder Rajesh with Status Disputed Ramesh Kumar filed a claim for Village Thiruvallur at Coordinates 12.9345, 80.2123 with Status Disputed.Patta holder Manoj owns land in Village Villupuram Coordinates 11.9401, 79.4861 Status Under Review")



colors = {"PATTA_HOLDER": "#F67DE3", "VILLAGE": "#7DF6D9", "COORDINATE": "#a6e22d", "CLAIM_STATUS": "#ff0000"} # Added CLAIM_STATUS color
options = {"colors": colors}

spacy.displacy.render(doc, style="ent", options=options, jupyter=True)